In [120]:
#import sys
#!{sys.executable} -m pip install futures
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
import pandas as pd
from time import time
from concurrent.futures import ThreadPoolExecutor
from scipy.optimize import curve_fit
with open("+The jungle book_Kipling.txt") as f:
    file=f.read()
 

#print(file)

In [1]:
import sys
!{sys.executable} -m pip install networkx



  Using cached networkx-2.5.1-py3-none-any.whl (1.6 MB)
  Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
  Attempting uninstall: decorator
    Found existing installation: decorator 5.0.9
    Uninstalling decorator-5.0.9:
      Successfully uninstalled decorator-5.0.9


In [144]:
from string import punctuation
def remove_punctuation(data):
    temp=[]
    for i in data:
        if i in punctuation:
            if i == "'" or i == '-'or i==' ':
                temp.append(i.lower())
        else:
            temp.append(i.lower())
    return "".join(temp)
data=remove_punctuation(file)

#print(data.split())



In [124]:

class Ngram (dict):
    def __init__ (self, iterable=None): # Ініціалізували наш розподіл як новий об'єкт класу, додаємо наявні елементи
        super(Ngram, self).__init__()
        self.F_i = 0 # число унікальних ключів в розподілі
        self.fa={}
        self.counts={}
        if iterable:
           self.update (iterable)
    def update (self, iterable): # Оновлюємо розподіл елементами з наявного итерируемого набору даних
        for item in iterable:
           if item in self:
               self[item]+=1
           else:
                self[item] = 1
                self.F_i+=1
    def hist(self):
        plt.bar(self.keys(),self.values())
        plt.show()

def make_dataframe(model,fmin):
    filtred_data=list(filter(lambda x:model[x].F_i >=fmin,model))
    data={"ngram":[],
          "F_i":np.empty(len(filtred_data),dtype=np.dtype(int))}
    
    #print(data['ngram'][0])
    for i,ngram in enumerate(filtred_data):
        data["ngram"].append(ngram)
        if ngram=="new_word":
            data["F_i"][i]=sum(model[ngram].bool)
            continue
        data["F_i"][i]=len(model[ngram].pos)
        
    return pd.DataFrame(data=data)
L=0#
V=0#

def make_markov_chain(data,order=1,split='word',):
    model=dict()
    
    
    
    
   
   
    
    
    
    if split=="symbol":
        temp=[]
        for word in data:
            for symbol in word: 
                temp.append(symbol)
        data=temp
    global L
    L=len(data)-order
    model['new_word']=Ngram()
    model['new_word'].bool=np.zeros(L)
    if order>1:
        for i in range(L):
            window = tuple (data[i: i+order])  # Додаємо в словник
            if window in model: # Приєднуємо до вже існуючого розподілу
                 model[window].update ([data[i+order]])
                 model[window].pos.append(i+1)
                 model[window].bool[i]=1
                 
                
            else:
                 model[window] = Ngram ([data[i+order]])
                 model[window].pos=[]
                 model[window].pos.append(i+1)
                 model[window].bool=np.zeros(L)
                 model[window].bool[i]=1
                 model['new_word'].bool[i]=1
    else:
        for i in range(L):
            if data[i] in model: # Приєднуємо до вже існуючого розподілу
                 model[data[i]].update ([data[i+order]])
                 model[data[i]].pos.append(i+order)
                 model[data[i]].bool[i]=1
                
            else:
                 model[data[i]] = Ngram ([data[i+order]])
                 model[data[i]].pos=[]
                 model[data[i]].pos.append(i+1)
                 model[data[i]].bool=np.zeros(L)
                 model[data[i]].bool[i]=1
                 model['new_word'].bool[i]=1
    global V
    V=len(model)
    return model
start=time()
fmin=0
order=2
split="word"
text="one fish two fish fish one fish two fish fish one fish two fish fish"
model=make_markov_chain(text.split(),order=order,split=split)
#model=make_markov_chain(data.split(),order=order,split=split)
#print(model)
#model
df=make_dataframe(model,fmin)
print("chain time:",time()-start)
#model.keys(),model.values())

#print(df)
#for ngram in model:
##    print(ngram)
 #   print(model[ngram].bool)
 #   print(model[ngram].pos)
print(L)
print(model['new_word'].bool)
#print(model['fish'].bool)
print(df)

chain time: 0.0009729862213134766
13
[1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
          ngram  F_i
0      new_word    5
1   (one, fish)    3
2   (fish, two)    3
3   (two, fish)    3
4  (fish, fish)    2
5   (fish, one)    2


# make markov chain graph

In [125]:
import networkx as nx
import plotly.graph_objects as go

g=nx.MultiGraph()
temp={}
for ngram in df['ngram']:
    g.add_node(ngram)
    temp[ngram[0]]=ngram
#print(temp.keys())   
for node in g.nodes():
    for i in model[node]:
        if i in temp:
            g.add_edge(node,temp[i],weight=model[node][i])
 

#nx.draw(g)
pos=nx.spring_layout(g)

edge_x = []
edge_y = []
for edge in g.edges():
    x0,y0=pos[edge[0]]
    x1,y1=pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in g.nodes():
    x,y=pos[node]
    node_x.append(x)
    node_y.append(y)
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))
node_adjacencies = []
node_text = []

for node, adjacencies in enumerate(g.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append(" ".join(adjacencies[0])+'<br>connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                  
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

# Calculate fa:
1.model[ngram].bool --> [0,0,0,0,0,1,1,0,1,...L]
2.windows(w,we,wh,wmax)

In [126]:
print("order:",order)
print("split by:",split)
print("L:",L)
print("V: ",V)
print("fmin:",fmin)
print("valid V:",len(df['ngram']))
print()
wmax=10#int(L/20)
w=2#int(wmax/10)
we=1
wh=w
print("w:",w)
print("wmax:",wmax)
print("we:",we)
print("wh:",wh)


opt ="no"
print("option:",opt)
#model['entropy'].bool

order: 2
split by: word
L: 13
V:  6
fmin: 0
valid V: 6

w: 2
wmax: 10
we: 1
wh: 2
option: no


## calculate distance (boundary condition)

In [127]:
@jit(nopython=True)
def nbc(positions):
    number_of_pos=len(positions)
    if number_of_pos ==1:
        return positions
    dt=np.empty(number_of_pos-1,dtype=np.uint8)
    for i in range(number_of_pos-1):
        dt[i]=positions[i+1]-positions[i]
    return dt

@jit(nopython=True)
def obc(positions,L):
    number_of_pos=len(positions)
    dt=np.empty(number_of_pos+1,dtype=np.uint8)
    dt[0]=positions[0]
    for i in range(number_of_pos-1):
        dt[i+1]=positions[i+1]-positions[i]
    dt[-1]=L-positions[-1]
    return dt

@jit(nopython=True)
def pbc(positions,L):
    number_of_pos=len(positions)
    dt=np.empty(number_of_pos,dtype=np.uint8)
    for i in range(number_of_pos-1):
        dt[i]=positions[i+1]-positions[i]
    dt[-1]=L-positions[-1]+L+positions[0]
    return dt


for index,ngram in enumerate(df['ngram']):
    print(str(index)+" of "+str(len(df['ngram'])),end="\r")
    if ngram=="new_word":
        model[ngram].dt=nbc(np.array(model[ngram].bool,dtype=np.uint8))
        continue
    model[ngram].dt=nbc(np.array(model[ngram].pos,dtype=np.uint8))
    #print(model[ngram].dt)


In [128]:
start=time()
temp_w=0


#print(df["ngram"])
@jit(nopython=True)
def s(window):
    suma=0
    for i in range(len(window)):
        suma+=window[i]
    return suma
@jit(nopython=True)
def mse(x):
    t=np.mean(x)
    st=np.mean(x**2)
    return np.sqrt(st-(t**2))

@jit(nopython=True)
def R(x):
    t=np.mean(x)
    if t == 0:
        return 0
    ts=np.mean(x**2)
    return np.sqrt(ts-(t**2))/t
    
@jit(nopython=True)
def fa(x,args):
    #print(w)
    wi,wsh,l=args
    count=np.empty(len(range(wi,l,wsh)),dtype=np.uint8)
    for index,i in enumerate(range(0,l-wi,wh)):
        count[index]=s(x[i:i+wi])
        
    return count,mse(count)
windows=list(range(w,wmax,we))
fa(np.zeros(5),(1,2,3))

def func(wind):
    model[ngram].counts[wind],model[ngram].fa[wind]=fa(model[ngram].bool,(wind,wh,L))

for index,ngram in enumerate(df['ngram']):
    print(str(index)+" of "+str(len(df['ngram'])),end="\r")
    with ThreadPoolExecutor() as e:
        e.map(func,windows)
        
 
@jit(nopython=True)
def fit(x,a,b):
    return a*x**b
temp_b=[]
temp_fi=[]
temp_R=[]
temp_cov=[]
for ngram in df['ngram']:  
    c,cov=curve_fit(fit,[*model[ngram].fa.keys()],[*model[ngram].fa.values()],maxfev=5000)
    model[ngram].a=c[0]
    model[ngram].b=c[1]
    temp_b.append(c[1])
    #temp_cov.append(cov)
    temp_fi.append(model[ngram].F_i)
    temp_R.append(R(np.array(model[ngram].dt)))
df['f_i']=temp_fi
df['R']=temp_R
#df["cov"]=temp_cov

df['alpha']=temp_b

    

    

    
"""
def secound_pool(ngram):
    return fa(model[ngram].bool,(temp_w,wmax,we,wh,L))

def first_pool(wind):
    #print(w)
    #print(list(df['ngram']))
    #print(model['entropy'])
    global temp_w
    temp_w=wind
    with ThreadPoolExecutor() as j:
        result = j.map(secound_pool,df['ngram'])
    for f,ngram in zip(result,df['ngram']):
        model[ngram].counts[wind]=f[0]
        model[ngram].fa[wind]=f[1]

    
with ThreadPoolExecutor() as e:
    windows=list(range(w,wmax,we))
    e.map(first_pool,windows)
"""


print("fa time:",time()-start)
df

fa time: 1.0499458312988281


,ngram,F_i,f_i,R,alpha
0,new_word,5,0,3.316625,0.149763
1,"(one, fish)",3,1,0.000000,-0.457687
2,"(fish, two)",3,1,0.000000,-0.023638
3,"(two, fish)",3,1,0.000000,-0.420500
4,"(fish, fish)",2,1,0.000000,-0.671812
5,"(fish, one)",2,1,0.000000,0.018539


# DFA

In [213]:
"""
notes 
w=2
wmax=4
wh=1
we=1
one fish two fish tree fish  

new_ngram(1)=[1,1,1,0,1,0]->{'one','fish','tow','fish','tree',"fish"}
dfa(2,3,4)
[1,1][ 1,1 ][1,1][1,1][1,1]->{[one,fish],[fish,two],[two,fish],[fish,tree],[tree,fish]}
[1,1,1][1,1,0][1,1,1][1,0,1]->{[one,fish,two],[fish,two,fish],[two,fish,tree],[fish,tree,fish]}
[1,1,1,0][1,1,0,1][1,1,1,0]->{[one,fish,two,fish],[fish,two,fish,tree],[two,fish,tree,fish]}



new_ngram(2)=[1,1,1,1,1]->[(one,fish),(fish,two),(two,fish),(fish,tree),(tree,fish)]
dfa(2,3,4)
[(one,fish),(fish,two)],[(fish,two),(two,fish)],[(two,fish),(fish,tree)],[(fish,tree),(tree,fish)]
->[1,1],[1,1],[1,1],[1,1]
[(one,fish),(fish,two),(two,fish)],[(fish,two),(two,fish),(fish,tree)],[(two,fish),(fish,tree),(tree,fish)]
->[1,1,1],[1,1,1],[1,1,1]
..


"""
data="one fish two fish tree fish"
#print(list(data[0:2]))
n=1
split="word"
l=0
def prepere_data(data,n,split):
    global l
    temp_data=[]
    if n==1:
        if split=="word":
        #data.replace(" ","space")
            l=len(data.split())
            return data.split()
        if split=='letter':
            data=remove_punctuation(data)
            for i in data:
                for j in i:
                    if j ==" ":
                        continue
                    temp_data.append(j)
            l=len(temp_data)
            return temp_data
        if split=='symbol':
            for i in data:
                for j in i:
                    if j ==" ":
                        temp_data.append("space")
                        continue
                    temp_data.append(j)
            l=len(temp_data)
            return temp_data
    if n>1:
        
        if split=="word":
            data=data.split()
            l=len(data)-n
            for i in range(l):
                window = tuple(data[i: i+n])
                temp_data.append(window)
            return temp_data
        if split=="letter":
            data=remove_punctuation(data.split())
            for i in data:
                for j in i:
                    temp_data.append(j)
            l=len(temp_data)-n
            data=temp_data
            temp_data=[]
            for i in range(l):
                window = tuple (data[i: i+n])
                temp_data.append(window)
            return temp_data
        if split=='symbol':
            temp_data=[]
            for i in data:
                for j in i:
                    if j==" ":
                        temp_data.append("space")
                        continue
                    temp_data.append(j)
            data=temp_data
            temp_data=[]
            l=len(data)-n
            for i in range(l):
                window=tuple(data[i:i+n])
                temp_data.append(window)
            return temp_data
            
            
                
                
            
                
            
        
    
data=prepere_data(data,n,split)
print(data)


w=2
wmax=5
wh=1
we=1
class newNgram():
    def __init__(self,data,wh,l):
        self.data=data
        self.count={}
        self.dfa={}
        self.wh,self.l=wh,l
    @jit(nopython=True)
    def dfa(data,args):
        wi,wh,l=args
        count=np.empty(len(range(wi,l,wh)),dtype=np.uint8)

        for index,i in enumerate(range(0,l-wi,wh)):
            temp_v=[]
            x=[]
            for ngram in data[i:i+wi]:
                if ngram in temp_v:
                    x.append(0)
                else:
                    temp_v.append(ngram)
                    x.append(1)
            count[index]=s(x)
        return count,mse(count)
    def func(self,w):
        
        self.count[w],self.dfa[w]=dfa(self.data,(w,self.wh,self.l))
    
    
        
windows=list(range(w,wmax,we))

print(windows)
print(l)
model={}
new_ngram=newNgram(data,wh,l)
##def func(wind):
 #   ngram.count[wind],ngram.dfa[wind]=dfa(data,(wind,wh,l))
    

  

    

    
    

with ThreadPoolExecutor() as e:
    e.map(new_ngram.func,windows)
new_ngram.dfa


['one', 'fish', 'two', 'fish', 'tree', 'fish']
[2, 3, 4]
6


{2: 0.0, 3: 0.4714045207910318, 4: 0.0}

# Graphs
    


In [129]:
import plotly.express as px
import plotly.graph_objects as go
#fig = px.scatter(
#    df, x='total_bill', y='tip', opacity=0.65,
#    trendline='ols', trendline_color_override='darkblue'
#)
#fig.show()
delta_f=[]
ngram=("new_word")
for w in model[ngram].fa.keys():
    delta_f.append(fit(w,model[ngram].a,model[ngram].b))

fig=go.Figure(
    [
        go.Scatter(x=[*model[ngram].fa.keys()],y=[*model[ngram].fa.values()],mode="markers"),
        go.Scatter(x=[*model[ngram].fa.keys()],y=delta_f)
        
    ]

)
fig.update_xaxes(type="log")
fig.show()
#print(len(np.array(model[ngram].pos)))
#print(len(model[ngram].bool))
fig1=go.Figure(
    [
        go.Scatter(x=np.array(range(L)),y=model[ngram].bool)
    ]
)
#fig1.update_layout(barmode='group')
fig1.show()

temp_hover=[]
for data in df['ngram']:
    
    temp_hover.append(" ".join(data))
    

fig2=go.Figure(
    [
        go.Scatter(x=df['R'],y=df['alpha'],text=temp_hover,mode="markers")
    ]
)
#temp_hover
#[*df['ngram']]
fig2.show()

# tests
order: 1
split by: word
L: 479154
V:  14262
fmin: 15
valid V: 1792

w: 4791
wmax: 47915
we: 1000
wh: 4791

# same window for all
fa time: 321.6836111545563 sec/(5,35 m)

# one by one 
fa time: 210.61292123794556 sec/ 3,5 m